In [ ]:
import pandas as pd
df=pd.read_csv(r"C:\Users\MY Laptop\Desktop\guvi_class\smart premium\train.csv")

In [2]:
z = (df['Annual Income']-df['Annual Income'].mean())/df['Annual Income'].std()
df['z score'] = z
df = df[(df['z score']<=3) & (df['z score']>=-3)]

df['Marital Status'] = df[['Marital Status']].fillna(df['Marital Status'].mode()[0])
df['Age'] = df[['Age']].fillna(df['Age'].median())
df['Number of Dependents'] = df[['Number of Dependents']].fillna(df['Number of Dependents'].median())
df['Previous Claims'] = df[['Previous Claims']].fillna(df['Previous Claims'].median())
df['Occupation'] = df[['Occupation']].fillna(df['Occupation'].mode()[0])
df['Health Score'] = df[['Health Score']].fillna(df['Health Score'].median())
df['Credit Score'] = df[['Credit Score']].fillna(df['Credit Score'].median())
df['Customer Feedback'] = df[['Customer Feedback']].fillna(df['Customer Feedback'].mode()[0])
df['Vehicle Age'] = df[['Vehicle Age']].fillna(df['Vehicle Age'].median())
df['Insurance Duration'] = df[['Insurance Duration']].fillna(df['Insurance Duration'].median())


In [ ]:
df['Gender']=df['Gender'].replace({'Female':0,'Male':1})
df['Smoking Status']=df['Smoking Status'].replace({'No':0,'Yes':1})
df=pd.get_dummies(df,columns=['Property Type',
'Marital Status',
'Education Level',
'Occupation',
'Location',
'Policy Type',
'Customer Feedback',
'Exercise Frequency'],
dtype='int')
df.drop(['Policy Start Date'],axis=1,inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
X=df.drop(['Premium Amount'],axis=1)
# X.isna().sum()
y=df['Premium Amount']
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [5]:
# params={
#     "n_estimators":25,
#     "max_depth":10,
#     "n_jobs":-1,
# }
# import mlflow
# mlflow.set_experiment('First exp')
# mlflow.set_tracking_uri("http://127.0.0.1:5000")

# with mlflow.start_run():
#     mlflow.log_params(params)
#     mlflow.log_metric('r2',r2_score(y_test,test_pred))
#     mlflow.sklearn.log_model(model,"Random Forest Regressor")

In [7]:
import xgboost as xgb
models=[(
    "RandomForestRegressor",
    {"n_estimators":40,"max_depth":5,"n_jobs":-1},
    RandomForestRegressor(),
    (x_train,y_train),
    (x_test,y_test)
    
),
(
    "XGBRegressor",
    {'objective':'reg:squarederror',"n_estimators":40,"max_depth":5,"n_jobs":-1},
    xgb.XGBRegressor(),
    (x_train,y_train),
    (x_test,y_test)

),
(
    "LinearRegression",
    {},
    LinearRegression(),
    (x_train,y_train),
    (x_test,y_test)

),
(
    "DecisionTreeRegressor",
    {},
    DecisionTreeRegressor(),
    (x_train,y_train),
    (x_test,y_test)

)
]

In [8]:
import numpy as np
reports=[]
for model_name,params,model,train_set,test_set in models:
    x_train=train_set[0]
    y_train=train_set[1]
    x_test=test_set[0]
    y_test=test_set[1]

    model.set_params(**params)
    model.fit(x_train,y_train)
    y_pred=model.predict(x_test)

    rmse=np.sqrt(mean_squared_error(y_test, y_pred))
    mae=mean_absolute_error(y_test,y_pred)
    r2=r2_score(y_train,train_pred)
    reports.append((model_name,rmse,mae,r2))

In [ ]:
import mlflow
mlflow.set_experiment('first  experiment')
mlflow.set_tracking_uri("http://127.0.0.1:5000")
for i,element in  enumerate(models):
    model_name=element[0]
    params=element[1]
    model=element[2]
    report=reports[i]
    
    with mlflow.start_run(run_name=model_name):
        mlflow.log_params(params)
        mlflow.log_metric('RMSE':report[1])
        mlflow.log_metric('MAE':report[2])
        mlflow.log_metric('R2':report[3])

        if 'XGBRegressor' in model_name:
             mlflow.xgboost.log_model(model,"model")
        else :
             mlflow.sklearn.log_model(model,"model")
    
    

2025/03/30 13:17:06 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\MYLAPT~1\AppData\Local\Temp\tmp9jt3j8q0\model\model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.5.1', 'cloudpickle==3.0.0']. Set logging level to DEBUG to see the full traceback. 
2025/03/30 13:17:09 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run RandomForestRegressor at: http://127.0.0.1:5000/#/experiments/844570940081017207/runs/0a120587c6ea4ec1832410492c863ea1
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/844570940081017207


C:\Users\MY Laptop\AppData\Roaming\Python\Python312\site-packages\xgboost\sklearn.py:1028: UserWarning: [13:17:28] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  self.get_booster().save_model(fname)
2025/03/30 13:19:41 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\MYLAPT~1\AppData\Local\Temp\tmpoft729jp\model, flavor: xgboost). Fall back to return ['xgboost==3.0.0']. Set logging level to DEBUG to see the full traceback. 
2025/03/30 13:19:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBRegressor at: http://127.0.0.1:5000/#/experiments/844570940081017207/runs/d4117223b59e44ada2803237d4e9bcab
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/844570940081017207


2025/03/30 13:20:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run LinearRegression at: http://127.0.0.1:5000/#/experiments/844570940081017207/runs/102b41281abd4af681ff8ccb004268f3
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/844570940081017207


2025/03/30 13:23:04 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\MYLAPT~1\AppData\Local\Temp\tmp5qbisufx\model\model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.5.1', 'cloudpickle==3.0.0']. Set logging level to DEBUG to see the full traceback. 
2025/03/30 13:23:05 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run DecisionTreeRegressor at: http://127.0.0.1:5000/#/experiments/844570940081017207/runs/34e328d516e54de89365d63ac73dd495
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/844570940081017207
